In [ ]:
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 401 kB 35.3 MB/s 
     |████████████████████████████████| 1.2 MB 57.7 MB/s 
     |████████████████████████████████| 19.7 MB 1.1 MB/s 
     |████████████████████████████████| 163 kB 62.3 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 1.6 MB 62.6 MB/s 
     |████████████████████████████████| 46 kB 4.9 MB/s 
     |████████████████████████████████| 981 kB 59.9 MB/s 
     |████████████████████████████████| 788 kB 68.1 MB/s 
     |████████████████████████████████| 5.3 MB 59.8 MB/s 
     |████████████████████████████████| 200 kB 75.2 MB/s 
     |████████████████████████████████| 62 kB 1.7 MB/s 
     |████████████████████████████████| 7.6 MB 61.1 MB/s 
  Created wheel for mpld3: filename=mpld3-0.3-py3-none-any.whl size=116702 sha256=cb3ea17aec0fd09edc2348cb0629b77e2d85beb6218d5be75d43fe8b34fc6421
  Stor

In [ ]:
from flair.models import TextClassifier
from flair.data import Sentence
classifier = TextClassifier.load('en-sentiment')
sentence = Sentence('you are average !')
classifier.predict(sentence)
# print sentence with predicted labels
print('Sentence above is: ', sentence.labels)

2022-10-31 10:48:18,686 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_4.pt not found in cache, downloading to /tmp/tmplghpe5_e


100%|██████████| 265512723/265512723 [00:21<00:00, 12259764.39B/s]

2022-10-31 10:48:41,014 copying /tmp/tmplghpe5_e to cache at /root/.flair/models/sentiment-en-mix-distillbert_4.pt


2022-10-31 10:48:41,695 removing temp file /tmp/tmplghpe5_e
2022-10-31 10:48:41,741 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Sentence above is:  ['Sentence: "you are average !"'/'NEGATIVE' (0.9981)]


In [ ]:
import pandas as pd
import tqdm
import re
import nltk

from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer 
from flair.data import Corpus
from flair.datasets import CSVClassificationCorpus
from flair.datasets import TREC_6
from flair.embeddings import TransformerDocumentEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from flair.data import Sentence

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/CE4045/nlp_vader_textblob_classified_data_updated.xlsx").drop_duplicates()


In [ ]:
trdf = df[df.label.notnull()]

In [ ]:
trdf = trdf[trdf.vader_preprocessing_text.notnull()]

In [ ]:
trdf = trdf[trdf.username != "Dan1el_San"]

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(trdf, train_size = 0.8, shuffle = True, stratify=trdf["label"])

In [ ]:
train.to_csv("./data/train.csv")
test.to_csv("./data/dev.csv")
test.to_csv("./data/test.csv")

In [ ]:
data_folder = './data/'

column_name_map = {15: "text", 5:"label_topic"}

corpus: Corpus = CSVClassificationCorpus(
    data_folder, 
    column_name_map,
    skip_header=True, 
    delimiter=',',
    label_type="label"
)

2022-10-20 14:04:18,870 Reading data from data
2022-10-20 14:04:18,872 Train: data/train.csv
2022-10-20 14:04:18,874 Dev: data/dev.csv
2022-10-20 14:04:18,875 Test: data/test.csv


In [ ]:
corpus.train[1]

Sentence: "manuel akanji is not part of the manchester city squad due to the birth of his new baby ." → 0.0 (1.0)

In [ ]:
label_dict = corpus.make_label_dictionary(label_type="label")

2022-10-20 14:04:25,943 Computing label dictionary. Progress:


3756it [00:01, 2704.78it/s]

2022-10-20 14:04:27,341 Dictionary created for label 'label' with 4 values: 1.0 (seen 1577 times), -1.0 (seen 1094 times), 0.0 (seen 1085 times)


In [ ]:
document_embeddings = TransformerDocumentEmbeddings('distilbert-base-uncased', fine_tune=True)

In [ ]:
test.label.value_counts()

 1.0    394
-1.0    274
 0.0    272
Name: label, dtype: int64

In [ ]:
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict, label_type="label")

In [ ]:
trainer = ModelTrainer(classifier, corpus)

In [ ]:
trainer.fine_tune('resources/taggers/question-classification-with-transformer',
                  learning_rate=5.0e-5,
                  mini_batch_size=4,
                  max_epochs=10,
                  )

2022-10-20 14:04:48,694 ----------------------------------------------------------------------------------------------------
2022-10-20 14:04:48,697 Model: "TextClassifier(
  (decoder): Linear(in_features=768, out_features=4, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
  (locked_dropout): LockedDropout(p=0.0)
  (word_dropout): WordDropout(p=0.0)
  (loss_function): CrossEntropyLoss()
  (document_embeddings): TransformerDocumentEmbeddings(
    (model): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0): TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_featur

100%|██████████| 235/235 [00:02<00:00, 93.14it/s]

2022-10-20 14:05:40,550 Evaluating as a multi-label problem: False
2022-10-20 14:05:40,568 DEV : loss 0.24089917540550232 - f1-score (micro avg)  0.5745


2022-10-20 14:05:41,166 BAD EPOCHS (no improvement): 4
2022-10-20 14:05:41,168 ----------------------------------------------------------------------------------------------------
2022-10-20 14:05:46,062 epoch 2 - iter 93/939 - loss 0.22058014 - samples/sec: 78.21 - lr: 0.000049
2022-10-20 14:05:50,973 epoch 2 - iter 186/939 - loss 0.21502278 - samples/sec: 77.95 - lr: 0.000049
2022-10-20 14:05:55,905 epoch 2 - iter 279/939 - loss 0.21234978 - samples/sec: 77.63 - lr: 0.000048
2022-10-20 14:06:00,746 epoch 2 - iter 372/939 - loss 0.21029714 - samples/sec: 79.03 - lr: 0.000048
2022-10-20 14:06:05,715 epoch 2 - iter 465/939 - loss 0.20671565 - samples/sec: 77.05 - lr: 0.000047
2022-10-20 14:06:10,661 epoch 2 - iter 558/939 - loss 0.20641266 - samples/sec: 77.38 - lr: 0.000047
2022-10-20 14:06:16,273 epoch 2 - iter 651/939 - loss 0.20759089 - samples/sec: 69.14 - lr: 0.000046
2022-10-20 14:06:21,232 epoch 2 - iter 744/939 - loss 0.20800294 - samples/sec: 77.37 - lr: 0.000046
2022-10-20 14

100%|██████████| 235/235 [00:02<00:00, 93.06it/s]

...

2022-10-20 14:13:43,315 Evaluating as a multi-label problem: False
2022-10-20 14:13:43,331 DEV : loss 1.0381135940551758 - f1-score (micro avg)  0.5553


2022-10-20 14:13:43,990 BAD EPOCHS (no improvement): 4
2022-10-20 14:13:44,762 ----------------------------------------------------------------------------------------------------
2022-10-20 14:13:44,764 Testing using last state of model ...


100%|██████████| 235/235 [00:02<00:00, 94.99it/s]

2022-10-20 14:13:47,259 Evaluating as a multi-label problem: False
2022-10-20 14:13:47,276 0.5553	0.5553	0.5553	0.5553
2022-10-20 14:13:47,277 
Results:
- F-score (micro) 0.5553
- F-score (macro) 0.5406
- Accuracy 0.5553

By class:
              precision    recall  f1-score   support

         1.0     0.6200    0.6624    0.6405       394
         0.0     0.4436    0.4338    0.4387       272
        -1.0     0.5652    0.5219    0.5427       274

    accuracy                         0.5553       940
   macro avg     0.5429    0.5394    0.5406       940
weighted avg     0.5530    0.5553    0.5536       940

2022-10-20 14:13:47,282 ----------------------------------------------------------------------------------------------------


{'test_score': 0.5553191489361702,
 'dev_score_history': [0.574468085106383,
  0.5095744680851064,
  0.5702127659574469,
  0.5819148936170213,
  0.5670212765957446,
  0.5680851063829787,
  0.5606382978723404,
  0.5574468085106383,
  0.5691489361702128,
  0.5553191489361702],
 'train_loss_history': [0.2619417859072756,
  0.20681954345951517,
  0.11218673676724163,
  0.05128887746798918,
  0.024863585842662413,
  0.014158669290335976,
  0.007562458485358221,
  0.0076489381075982455,
  0.003596867940602266,
  0.002521112645314288],
 'dev_loss_history': [0.24089917540550232,
  0.296430766582489,
  0.39664363861083984,
  0.5904138684272766,
  0.7090975046157837,
  0.8726459741592407,
  0.9577162861824036,
  0.9488041996955872,
  1.0004475116729736,
  1.0381135940551758]}